In [ ]:
#https://github.com/Kulbear/deep-learning-coursera/blob/master/Convolutional%20Neural%20Networks/Keras%20-%20Tutorial%20-%20Happy%20House%20v1.ipynb
# Data Augmentation Keras ImageDataGenerator https://keras.io/preprocessing/image/
# Regularisation: Dropout, L2
# Optmisation: Batch normalisation, optmiser choice
# Parameter tuning: Architecture
#https://helpx.adobe.com/hk_en/photoshop/using/image-size-resolution.html#resampling
#Each pixel in image is 1 byte (8 bit) sized in each colour channel

In [18]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D, BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils, np_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import * 
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import pandas as pd
import PIL
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [19]:
def read_data(path):
    train=pd.read_csv(path+'/train.csv')
    test=pd.read_csv(path+'/test.csv')
    return train,test

train,test=read_data('./Datasets')

def generate_input(path,image_ids):
    for id in image_ids:
        img=image.load_img(path+id,target_size=(64,64))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        yield x
    
X=np.concatenate([feat for feat in generate_input('./Datasets/Train/',train['ID'])])
#X_test=np.concatenate([feat for feat in generate_input('./Datasets/Test/',test['ID'])])

lb = preprocessing.LabelBinarizer()
y=lb.fit_transform(train['Class'])
#np_utils.to_categorical(train['Class'], 3)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
model=Sequential()
model.add(ZeroPadding2D((3,3),input_shape=X_train.shape[1:]))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('softmax'))

In [25]:
opt=keras.optimizers.rmsprop(lr=0.00001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test),shuffle=True)

Train on 13337 samples, validate on 6569 samples
Epoch 1/20
13337/13337 [==============================] - 624s 47ms/step - loss: 3.5048 - acc: 0.4397 - val_loss: 1.4841 - val_acc: 0.4608
Epoch 2/20
13337/13337 [==============================] - 625s 47ms/step - loss: 2.3224 - acc: 0.4715 - val_loss: 1.1515 - val_acc: 0.4978
Epoch 3/20
13337/13337 [==============================] - 625s 47ms/step - loss: 1.7611 - acc: 0.4943 - val_loss: 1.0301 - val_acc: 0.5258
Epoch 4/20
13337/13337 [==============================] - 624s 47ms/step - loss: 1.4467 - acc: 0.5053 - val_loss: 0.9631 - val_acc: 0.5427
Epoch 5/20
13337/13337 [==============================] - 626s 47ms/step - loss: 1.2342 - acc: 0.5315 - val_loss: 0.9226 - val_acc: 0.5668
Epoch 6/20
13337/13337 [==============================] - 626s 47ms/step - loss: 1.0876 - acc: 0.5526 - val_loss: 0.9199 - val_acc: 0.5604
Epoch 7/20
13337/13337 [==============================] - 542s 41ms/step - loss: 1.0073 - acc: 0.5672 - val_loss: 0.8

KeyboardInterrupt: 

In [26]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

6569/6569 [==============================] - 54s 8ms/step
Test loss: 0.8347945712820946
Test accuracy: 0.6439336276812937
